In [ ]:
# get voronoi boundaries

# get numpy array of image
#   - get centers of nuclei in numpy array
#   - prediction.boxes.xywh.numpy()
#       - export to csv
#           - how do you export to csv?


In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
from IPython.display import clear_output
import socket
import time
from datetime import datetime
import shutil
from random import randint
import re
import imagecodecs
import tifffile
import tensorflow as tf
from scipy.spatial import Voronoi, cKDTree
from matplotlib.path import Path
from rtree import index
from ctypes.wintypes import INT
import time
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import torch, torchvision
import colorsys

print("torchvision: ", torchvision.__version__)
print('CUDA available:', torch.cuda.is_available())

torchvision:  0.17.2
CUDA available: True


In [2]:
# is this a segmentation model
yolo_model_path = r"S:\Phys\FIV925 XSection\Datasets\Axons vs SCs\01i5\YO 384 0530 MAXI\map75=0855192 YOLOv8n-seg map75=081193 idx=4 ep=24 btch=32 rnd=2840511\weights\best.pt"

model = YOLO(yolo_model_path)

In [3]:
def get_color_by_id(point_region_id, total_ids):
    hue = point_region_id / total_ids # Scale the hue by the number of unique IDs, wrapping around the hue circle
    saturation = 0.9; value = 0.9  # Keep saturation and value high for bright colors
    rgb = colorsys.hsv_to_rgb(hue, saturation, value)
    r, g, b = (int(c * 255) for c in rgb)  # Convert to 0-255 scale for RGB colors
    return (r << 16) | (g << 8) | b  # Combine into a single integer

def draw_polygons(draw, polygons):
    for i, polygon in enumerate(polygons):
        if len(polygon) < 2:
            continue
        # color = get_color_by_id(i, len(polygons))
        draw.polygon(polygon, fill=i, outline='white')
    
    

In [ ]:
tile_size = 384
iteration = 0
scale_factor = 1
dataset = "Human Lung Cancer"
base_path = r"R:\FIVE\EXP\FIV925\Additional Datasets"
ome_tiff_path = f"{base_path}\\{dataset}\\morphology_mip.ome.tif"

with tifffile.TiffFile(ome_tiff_path) as tif:
    image = tif.asarray()
    # image_dapi = image[2,:, :] # DAPI channel
    image_data = image[::2, ::2] # downsample by half

    # normalize to 8 bit
    max_val_16bit = np.max(image_data)
    image_scaled = (image_data / max_val_16bit) * 255
    image_8bit = image_scaled.astype(np.uint8)

    # create image and resize
    og_image = Image.fromarray(image_8bit, 'L')
    new_width = int(og_image.width * scale_factor); new_height = int(og_image.height * scale_factor)
    image_resized = og_image.resize((new_width, new_height), Image.LANCZOS)
    image = np.array(image_resized)

    image_height, image_width = image_resized.size

    big_array = np.zeros((image_resized.height + 384, image_resized.width + 384))
    list_xywh = []

    # as we're making predictions, we want a big array that's meant to 'contain' the whole image
    #   then insert the predictions into that big array
    # so here is where we go through the image as tiles and make the predictions
    tile_id = 0
    for i in range(0, image_height, 384): # section_miny, section_maxy
        for j in range(0, image_width, 384): # section_minx, section_maxx
            iteration += 1
            tile = image[i:i+tile_size, j:j+tile_size]
            
            tile_id += 1
            
            if tile.shape[0] == 0 or tile.shape[1] == 0:
                continue  # Skip tiles that have no width or height
            tile_as_img = Image.fromarray(tile, 'L')
            predictions = model.predict(tile_as_img, show=False, max_det=3000)
            
            result = predictions[0]
            # draw masks onto blank image
            if result.masks is not None:
                polygons = result.masks.xy
            else:
                continue
            draw_tile = ImageDraw.Draw(tile_as_img)
            draw_polygons(draw_tile, polygons)
            image_array = np.array(tile_as_img)
            image_array[image_array != 0] += tile_id
            
            tile_id += len(polygons)

            # tile = np.array(tile_as_img)
            big_array[i:i+tile.shape[0], j:j+tile.shape[1]] = image_array
            
            print("iteration: ", iteration)
            
# Save the array with a header and formatted floating-point numbers
# np.savetxt(r"R:\FIVE\EXP\FIV925\Additional Datasets\Human Lung Cancer\array.csv", big_array, delimiter=",", fmt='%.2f')    

In [ ]:
from skimage.measure import label 

big_array_unique = big_array 

labeled_array = label(big_array_unique, connectivity=1)

In [ ]:
np.savetxt(r"R:\FIVE\EXP\FIV925\Additional Datasets\Human Lung Cancer\array.csv", big_array, delimiter=",", fmt='%.2f')  